In [ ]:
import davitpy.pydarn.sdio.dbUtils as db
from davitpy import rcParams
import datetime as dt
import os
import sys
module_path = os.path.abspath(os.path.join('../py/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import ae

In [ ]:
sYear = 2016
eYear = 2016
res = 1

In [ ]:
mongoData = db.getDataConn(username="sd_dbwrite",password="more_dbwrite",\
                dbAddress="sd-work9.ece.vt.edu:27017",dbName='gme',collName='ae')
#set up all of the indices
mongoData.ensure_index('time')
mongoData.ensure_index('ae')
mongoData.ensure_index('al')
mongoData.ensure_index('au')
mongoData.ensure_index('ao')
mongoData.ensure_index('res')

In [ ]:
for yr in range(sYear,eYear+1):
        #1 day at a time, to not fill up RAM
        templist = ae.readAeWeb(dt.datetime(yr,1,1),dt.datetime(yr,1,1)+dt.timedelta(days=366),res=res)
        if(templist == None): continue
        for rec in templist:
            #check if a duplicate record exists
            qry = mongoData.find({'$and':[{'time': rec.time}, {'res': rec.res}]})
            print rec.time
            tempRec = rec.toDbDict()
            cnt = qry.count()
            #if this is a new record, insert it
            if(cnt == 0): mongoData.insert(tempRec)
            #if this is an existing record, update it
            elif(cnt == 1):
                print 'foundone!!'
                dbDict = qry.next()
                temp = dbDict['_id']
                dbDict = tempRec
                dbDict['_id'] = temp
                mongoData.save(dbDict)
            else:
                print 'strange, there is more than 1 AE record for ' + rec.time
del templist